# General material models

In [1]:
import numpy as np

from sigmaepsilon.solid.material import ElasticityTensor
from sigmaepsilon.math.linalg import ReferenceFrame
from sigmaepsilon.solid.material.utils import elastic_stiffness_matrix
from sigmaepsilon.math.logical import isposdef, issymmetric
from sigmaepsilon.solid.material import (
    ElasticityTensor,
    LinearElasticMaterial,
    HuberMisesHenckyFailureCriterion,
)

hooke = elastic_stiffness_matrix(E=210000, NU=0.3)
assert isposdef(hooke)
assert issymmetric(hooke, 1e-8)

yield_strength=355.0

frame = ReferenceFrame(dim=3)
stiffness = ElasticityTensor(hooke, frame=frame, tensorial=False)
failure_model = HuberMisesHenckyFailureCriterion(yield_strength=yield_strength)
material = LinearElasticMaterial(stiffness=stiffness, failure_model=failure_model)

f:\GitHub\SigmaEpsilon\sigmaepsilon.solid.material\.solid.material\lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [2]:
material.calculate_strains(stresses=np.array([yield_strength, 0.0, 0.0, 0.0, 0.0, 0.0]))

<xarray.DataArray (index: 1, component: 6)>
array([[ 0.00169048, -0.00050714, -0.00050714,  0.        ,  0.        ,
         0.        ]])
Coordinates:
  * index      (index) int32 0
  * component  (component) <U3 'EXX' 'EYY' 'EZZ' 'EYZ' 'EXZ' 'EXY'

In [3]:
material.utilization(stresses=np.random.rand(10, 6))

<xarray.DataArray (index: 10)>
array([0.00363546, 0.00533677, 0.00432855, 0.00724814, 0.0037595 ,
       0.00271517, 0.00511495, 0.00580621, 0.00416432, 0.00340729])
Coordinates:
  * index    (index) int32 0 1 2 3 4 5 6 7 8 9

In [4]:
material.calculate_stresses(strains=np.random.rand(6)).shape

(1, 6)

In [5]:
material.calculate_stresses(strains=np.random.rand(10, 6)).shape

(10, 6)

In [6]:
stresses=np.array([yield_strength, 0.0, 0.0, 0.0, 0.0, 0.0])
strains = material.calculate_strains(stresses=stresses)
material.utilization(stresses=stresses) * 100

<xarray.DataArray ()>
array(100.)

In [7]:
strains = material.calculate_strains(stresses=np.eye(6) * yield_strength)
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses)

<xarray.DataArray (index: 6)>
array([1.        , 1.        , 1.        , 1.73205081, 1.73205081,
       1.73205081])
Coordinates:
  * index    (index) int32 0 1 2 3 4 5

In [8]:
strains = material.calculate_strains(stresses=np.array([10*yield_strength, 10*yield_strength, 10*yield_strength, 0.0, 0.0, 0.0]))
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses)

<xarray.DataArray ()>
array(2.56195691e-15)

Utilization for a single state of strain:

In [9]:
strains = 2*np.random.rand(6)/1000
strains

array([0.00054175, 0.00064456, 0.00126496, 0.00167429, 0.00155765,
       0.00107135])

In [10]:
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses) * 100

<xarray.DataArray ()>
array(104.18523758)

Utilizations for several states with bulk evaluation:

In [11]:
material.utilization(stresses=2*np.random.rand(10, 6)/1000) * 100

<xarray.DataArray (index: 10)>
array([0.0013057 , 0.00063336, 0.00094936, 0.00119699, 0.00093216,
       0.00107443, 0.00083428, 0.00091125, 0.00103994, 0.00080839])
Coordinates:
  * index    (index) int32 0 1 2 3 4 5 6 7 8 9

In [12]:
material.utilization(stresses=2*np.random.rand(10, 6)/1000).shape

(10,)

In [13]:
material.utilization(stresses=tuple(2*np.random.rand(10)/1000 for _ in range(6))) * 100

<xarray.DataArray (index: 10)>
array([0.00110169, 0.00097445, 0.00063473, 0.00065579, 0.00100737,
       0.00059351, 0.00081348, 0.00065195, 0.00094461, 0.00122594])
Coordinates:
  * index    (index) int32 0 1 2 3 4 5 6 7 8 9